# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

- Для решения этой задачи будем умножать признаки на обратимую матрицу. 
- Ответим на вопрос: изменится ли качество линейной регрессии?


a. Изменится. Если да, то приведем примеры матриц.

b. Не изменится. Если да, то укажем, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
- Поскольку качество линейной регрессии не изменится, то предложим алгоритм преобразования данных для решения задачи. и обоснуем, почему же качество линейной регрессии не поменяется.
- Запрограммируем этот алгоритм, применив матричные операции. 
- Проверим, что качество линейной регрессии из sklearn не отличается до и после преобразования (применим метрику R2)

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns

# import random
# import math
from scipy import stats

# import matplotlib.pyplot as plt
# from pandas.plotting import scatter_matrix

# #date and time 
# import datetime 
# from datetime import datetime, date

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression  
# from sklearn.linear_model import LogisticRegression
# from sklearn.neural_network import MLPRegressor 
# import xgboost

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from sklearn.dummy import DummyClassifier
# from sklearn.utils import shuffle
# from sklearn.model_selection import GridSearchCV 

from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
# from sklearn.metrics import precision_recall_curve, roc_curve

pd.set_option('display.max_columns', 40) #макс кол-во колонок в выводимых данных 
pd.set_option('display.max_rows', 20) #макс кол-во строк в выводимых данных
pd.set_option('display.width', 80) #макс кол-во символов в строке

%matplotlib inline

import warnings
warnings.simplefilter('ignore')

pd.options.mode.chained_assignment = None

#from pyspark.sql import SparkSession

In [2]:
try: 
    df=pd.read_csv('/datasets/insurance.csv')
except: 
    df=pd.read_csv('datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
df.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


In [7]:
def unique_rows (data):
    for column in data: 
        print ('Уникальные значения', column)
        print (data[column].unique())
        print ('Пропуски, количество')
        print (data.isna().mean())
        print ()
unique_rows(df)

Уникальные значения Пол
[1 0]
Пропуски, количество
Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

Уникальные значения Возраст
[41. 46. 29. 21. 28. 43. 39. 25. 36. 32. 38. 23. 40. 34. 26. 42. 27. 33.
 47. 30. 19. 31. 22. 20. 24. 18. 37. 48. 45. 44. 52. 49. 35. 56. 65. 55.
 57. 54. 50. 53. 51. 58. 59. 60. 61. 62.]
Пропуски, количество
Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

Уникальные значения Зарплата
[49600. 38000. 21000. 41700. 26100. 41000. 39700. 38600. 49700. 51700.
 36600. 29300. 39500. 55000. 43700. 23300. 48900. 33200. 36900. 43500.
 36100. 26600. 48700. 40400. 38400. 34600. 34800. 36800. 42200. 46300.
 30300. 51000. 28100. 64800. 30400. 45300. 38300. 49500. 19400. 40200.
 31700. 69200. 33100. 31600. 34500. 38700. 39600. 42400. 34900. 30500.
 24200. 49900. 14300. 47000. 44800. 43800. 427

In [8]:
df['Возраст']=df['Возраст'].astype('int')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   int32  
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(1), int32(1), int64(3)
memory usage: 175.9 KB


Данные уже подготовлены для использования, не имеют аномалий. Можно переходить к заданию. 

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Проверим формулу, по которой работает линейная регрессия, используемая из Scikit Learn, на предмет соответствия данной нам ячейкой выше формуле обучения. 

In [10]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [11]:
zeros = np.ones((features.shape[0], 1))
X = np.concatenate((zeros, features), axis=1)
y = target
w = (np.linalg.inv(X.T @ X)) @ X.T @ y
w[1:]

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

In [12]:
model_lr = LinearRegression()
model_lr.fit(features, target)
model_lr.coef_

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

In [13]:
w[1:] - model_lr.coef_

array([-2.88137569e-15,  7.63278329e-17, -6.25213547e-18, -1.07552856e-16])

Как мы видим, найденные коэффициенты совпадают. Перейдем к заданному нам вопросу. 

__Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?__<div>
_Наш ответ: не изменится._ 

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} E X^T y
$$
$$
w' = P^{-1}w
$$

Если рассматривать передачу данных третьим лицам, то к преимуществам данного подхода можно отнести, в первую очередь, ощутимую экономию ресурсов по сравнению с некоторыми известными алгоритмами шифрования, в особенности это касается ассиметричных алгоритмов с достаточно длинными ключами (AES, RSA), которые требуют много процессорного времени.

К главному недостатку подхода можно отнести увеличение сложности проверки на вырожденность преобразования с ростом количества признаков в выборке. В рассмотренном случае необходимо подобрать подходящую матрицу размера 4×4 . Однако, если признаки будут исчисляться тысячами, что в теории всегда надо положить, то расчет соответствующих определителей может быть крайне ресурсозатратным. 

## Алгоритм преобразования

По факту персональных данных в нашем датафрейме не содержится, для того, чтобы данные стали непонятными для человека, их достаточно пропустить через Scaler, но поскольку здесь наша задача - освоить использование матриц, то не задаваясь вопросом о качестве такого "шифрования" данных будем умножать матрицу признаков на некую обратимую матрицу, генерируемую случайным образом. 

Пусть это будет матрица _Y_. Поступим с ней по следующей схеме:
- Получим эту матрицу. 
- Проверим на обратимость и найдем детерминант. 
- Получим матрицу преобразованных признаков (Z=XY)
- Применим алгоритм на преобразованных признаках Z.

Матрица Y должна иметь необходимую размерность (n*n), где n - количество признаков. 

Т.е. размерность Z будет иметь та же, что X. 

Обратная матрица Y будет существовать только для квадратных невырожденных матриц, т.е. тех, у кого определитель не ноль.

## Проверка алгоритма

Исследуем качество модели линейной регрессии без преобразования с исходными признаками и с масштабированными признаками. Далее преобразуем признаки, используя матрицу, о которой говорили выше, и снова точно так же проверим ее качество: с "исходными" преобразованными признаками и с этими же признаками, пропущенными через Scaler.

In [14]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=777777)

In [15]:
lr_model=LinearRegression()
lr_model.fit(features_train, target_train)
R2_first_score = r2_score(target_test, lr_model.predict(features_test))
R2_first_score

0.42678762016805494

(для более-менее приемлемых моделей предполагается, что коэффициент детерминации должен быть хотя бы не меньше, чем 0.5)

Вектор w:

In [16]:
lr_model.coef_

array([ 9.09778557e-03,  3.52956349e-02,  1.36753369e-07, -1.14043518e-02])

Используем StandardScaler и Pipeline (правда, в этом случае, мы не увидим коэффициенты регрессии):

In [17]:
lr_model_scaled=LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", lr_model_scaled)])
pipeline.fit(features_train, target_train)
R2_second_score = r2_score(target_test, pipeline.predict(features_test))
R2_second_score

0.4267876201680567

In [18]:
R2_first_score - R2_second_score

-1.7763568394002505e-15

Разница в R2 очень мала и связана, очевидно, с точностью хранения данных в float.

Опишем нашу функцию "кодирования информации":

In [19]:
def crypto_func (features): 
    crypted_features = features
    n = features.shape[1]
    np.random.seed(7777777)
    crypted_matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(crypted_matrix)
    while det == 0: 
        np.random.seed(7777777)
        crypted_matrix = np.random.randint(1, 10, (n,n))
        det = np.linalg.det(crypted_matrix)
    crypted_features = crypted_features @ crypted_matrix
    return crypted_features, crypted_matrix

До применения crypto_func данные выглядят так:

In [20]:
features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600.0,1
1,0,46,38000.0,1
2,0,29,21000.0,0
3,0,21,41700.0,2
4,1,28,26100.0,0


In [21]:
features, crypted_matrix = crypto_func(features)

In [22]:
features.head()

,0,1,2,3
0,446658.0,49772.0,49901.0,397052.0
1,342282.0,38187.0,38330.0,304280.0
2,189174.0,21116.0,21203.0,168174.0
3,375438.0,41790.0,41863.0,333734.0
4,235074.0,26217.0,26302.0,208970.0


In [23]:
crypted_matrix

array([[6, 5, 6, 2],
       [6, 4, 7, 6],
       [9, 1, 1, 8],
       [6, 3, 8, 4]])

Очевидно, что визуально идентифицировать смысл данных теперь нельзя. 

Снова поделим данные и проверим качество модели LinearRegression.

In [24]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=777777)

In [25]:
model_lr_crypted = LinearRegression()
model_lr_crypted.fit(features_train, target_train)
R2_third_score = r2_score(target_test, model_lr_crypted.predict(features_test))
R2_third_score

0.4267876201674071

In [26]:
model_lr_crypted.coef_

array([-0.01219729,  0.01748968, -0.00491103,  0.01214964])

Вновь используем StandardScaler и Pipeline:

In [27]:
lr_model_crypted_scaled=LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", lr_model_crypted_scaled)])
pipeline.fit(features_train, target_train)
R2_fourth_score = r2_score(target_test, pipeline.predict(features_test))
R2_fourth_score

0.42678762016809124

In [28]:
R2_third_score - R2_fourth_score

-6.841194277740215e-13

Значение также очень мало. Как и разница между R2 по результатам масштабированных данных и масштабированных "зашифрованных" данных.

In [29]:
R2_second_score - R2_fourth_score

-3.452793606584237e-14

Для наглядности соберем итоговый датафрейм с R2:

In [30]:
R2_score_df = pd.DataFrame(data= [R2_first_score, 
                                  R2_second_score, 
                                  R2_third_score, 
                                  R2_fourth_score], 
                     columns=['R2'], 
                     index=['LinearRegression', 
                            'LinearRegression Scaled', 
                            'LinearRegression Cryped', 
                            'LinearRegression Cryped Scaled'])
R2_score_df

,R2
LinearRegression,0.426788
LinearRegression Scaled,0.426788
LinearRegression Cryped,0.426788
LinearRegression Cryped Scaled,0.426788


__Вывод:__

- Данные можно зашифровать от визуального распознования человеком, имея подходящую матрицу (при использовании модели линейной регрессии). 

- Получен отрицательный ответ на вопрос о том, изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу. 